**Name:** Kamran Ansari

**Reg No:** 22MCA0223





In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
# https://www.kaggle.com/datasets/kazanova/sentiment140

!gdown https://drive.google.com/uc?id=1T-T9GpRR53LruxTFvapYTXmzN7ImHOt8&export=download

Downloading...
From: https://drive.google.com/uc?id=1T-T9GpRR53LruxTFvapYTXmzN7ImHOt8
To: /content/sentiment.zip
100% 84.9M/84.9M [00:01<00:00, 45.8MB/s]


In [ ]:
!unzip /content/sentiment
!mv training.1600000.processed.noemoticon.csv data.csv

Archive:  /content/sentiment.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [ ]:
df = pd.read_csv('/content/data.csv', delimiter=',', encoding='latin-1', header=None, names=["target", "ids", "date", "flag", "user", "text"])
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df.drop(["ids", "date", "flag", "user"], axis = 1, inplace = True)
df.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   text    1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [ ]:
X = df.text
Y = df.target
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)
Y

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)

In [ ]:
from keras.utils import pad_sequences

max_words = 1000
max_len = 150

tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X_train)

sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences, maxlen = max_len)

In [ ]:
def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 50, input_length=max_len)(inputs)

    layer = LSTM(64)(layer)

    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)

    layer = Dropout(0.2)(layer)

    layer = Dense(128, name='FC3')(layer)
    layer = Activation('relu')(layer)

    layer = Dense(64, name='FC4')(layer)
    layer = Activation('relu')(layer)

    layer = Dropout(0.3)(layer)

    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)

    model = Model(inputs=inputs, outputs=layer)
    return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_10 (Embedding)    (None, 150, 50)           50000     
                                                                 
 lstm_10 (LSTM)              (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_41 (Activation)  (None, 256)               0         
                                                                 
 dropout_14 (Dropout)        (None, 256)               0         
                                                                 
 FC3 (Dense)                 (None, 128)               328

In [ ]:
model.fit(sequences_matrix, Y_train, batch_size=128, epochs=10,
          validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0001)])

Epoch 1/10
8500/8500 [==============================] - 83s 10ms/step - loss: 0.4712 - accuracy: 0.7750 - val_loss: 0.4531 - val_accuracy: 0.7847
Epoch 2/10
8500/8500 [==============================] - 77s 9ms/step - loss: 0.4487 - accuracy: 0.7888 - val_loss: 0.4420 - val_accuracy: 0.7905
Epoch 3/10
8500/8500 [==============================] - 77s 9ms/step - loss: 0.4386 - accuracy: 0.7946 - val_loss: 0.4380 - val_accuracy: 0.7929
Epoch 4/10
8500/8500 [==============================] - 74s 9ms/step - loss: 0.4303 - accuracy: 0.7994 - val_loss: 0.4309 - val_accuracy: 0.7971
Epoch 5/10
8500/8500 [==============================] - 74s 9ms/step - loss: 0.4237 - accuracy: 0.8033 - val_loss: 0.4283 - val_accuracy: 0.7984
Epoch 6/10
8500/8500 [==============================] - 75s 9ms/step - loss: 0.4184 - accuracy: 0.8064 - val_loss: 0.4270 - val_accuracy: 0.8003
Epoch 7/10
8500/8500 [==============================] - 74s 9ms/step - loss: 0.4138 - accuracy: 0.8086 - val_loss: 0.4275 - val_a

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)

7500/7500 [==============================] - 40s 5ms/step - loss: 0.6205 - accuracy: 0.7694


In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.621
  Accuracy: 0.769
